In [1]:
from googlesearch import search
import pandas as pd
import numpy as np

# 2ème scraping à partir du noms des jeux dans le dataset original
Certains jeux du dataset original n’apparaissent pas le dataset issu du 1er scraping. <br>
1) Après avoir créé une liste des jeux manquants, utilisation de la bibliothèque googlesearch pour récupérer les liens Gamekult de ces jeux<br>
2) Pour chaque lien valide (liens Gamekult), application de la 1ère méthode de scraping

### 1) Scrap googlesearch des jeux manquants

In [2]:
#Import du dataset original et du dataset issu du 1er scraping pour création de la liste des jeux manquants
df_origin = pd.read_csv('../data/vgsales.csv')
df_GK = pd.read_csv('../data/01_GameSpy_Scraping_gk_01.csv', index_col=0)

list_game_missing = sorted([x for x in df_origin['Name'].unique() if x not in df_GK['Name'].unique()])

In [3]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [5]:
#Rajout du mot gamekult pour cibler plus facilement les liens
list_a_scraper = []
for game in list_game_missing:
    rajout = game + " gamekult"
    list_a_scraper.append(rajout)

#Récupération des liens de ces jeux
dict_url = {}
for i in range(0,len(list_a_scraper)):
    for url in search(list_a_scraper[i], 
                      tld='com',
                      lang='fr',
                      start=0,
                      stop=2
                   ):
        #Validation des liens Gamekult uniquement
        if 'gamekult' in url:
            dict_i = {
                list_a_scraper[i]: url
            }
        dict_url = dict(dict_url, **dict_i)
        
df_url = pd.DataFrame.from_dict(dict_url, columns=['Name', 'url'])
df_url['Name'] = df_url['Name'].replace(' gamekult', '')

#WARNING: Pour fonctionner, le range de i doit être divisé en plusieurs séquences de 900 car sinon le code plante

0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
41
41
42
42
43
44
44
45
45
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
74
74
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
84
84
85
85
86
86
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
149
151
152
152
153
153
154
154
155
155
156
156
157
157
158
158
159
159
160
160
161

RemoteDisconnected: Remote end closed connection without response

### 2) Scrap des informations dans chaque lien Gamekult récupéré

In [ ]:
#Fonction pour récupérer les infos nécessaires dans une page de test
#Cette fonction prend en argument une url et renvoie un dictionnaire contenant les informations scrapées

def extract_data(url):
    page = requests.get(url)
    parsedPage = BeautifulSoup(page.content, 'html.parser')

            
    # Scrap : note joueurs
    note_joueur = parsedPage.select(".gk__user-note__md span")
    note_joueur = str(note_joueur)
    if len(parsedPage.select(".gk__user-note__md span")) != 0:
        note_joueur = note_joueur.split('<')
        note_joueur_split = []
        for i in note_joueur:
            note_joueur_split += i.split('>')
        if len(note_joueur_split) < 3 :
            note_joueur_data = None
        else:
            note_joueur_data = note_joueur_split[2].strip()
    else:
        note_joueur_data = None
     
    
    # Scrap : nb avis
    nb_avis = parsedPage.select(".us__reviews-h__opinion-count a")
    nb_avis = str(nb_avis)
    if len(parsedPage.select(".us__reviews-h__opinion-count a")) != 0:
        nb_avis = nb_avis.split('(')
        nb_avis = nb_avis[1].split(')')
        nb_avis = nb_avis[0].split(' ')
        nb_avis = nb_avis[2]
    else:
        nb_avis = None
    
    
    # Scrap : note GK
    note_GK = parsedPage.select(".gk__staff-note__inset span")
    note_GK = str(note_GK)
    note_GK = note_GK.split('<')
    note_GK_split = []
    for i in note_GK:
        note_GK_split += i.split('>')
    if len(note_GK_split) < 3 :
        note_GK_data = None
    else:
        note_GK_data = note_GK_split[2].strip()
    
    
    # Scrap zone infos
    zt = parsedPage.find_all('th', {'class':'gk__data-table__item'})
    zc = parsedPage.find_all('td', {'class':'gk__data-table__value gk__text-left'})
    list_info_titre = []
    list_info_contenu = []
    for th in zt:
        list_info_titre.append(th.text)
    for td in zc:
        list_info_contenu.append(td.text)
    
    
    
    data = {
        'url' : url,
        'Note_joueurs' : note_joueur_data,
        'Nb_avis' : nb_avis,
        'Note_GK' : note_GK_data,
        'Zone_titre' : list_info_titre,
        'Zone_contenu' : list_info_contenu
        }
    
    return data

In [ ]:
#Création du dataframe 
new_data_full = []

for url in list(dict_url.values()):
    scrap = extract_data(url)
    new_data_full.append(scrap)
    
new_data_pd = pd.DataFrame.from_dict(new_data_full)
df = df_url.merge(right=new_data_pd, on='url', how='left')

In [ ]:
#Cleaning de zone_titre et zone_contenu où se trouvent les infos concernant le distributeur, le développeur, l'éditeur, la franchise de chaque jeu

zone_dico = []

for i in range(0,df.shape[0]):
    a = df['Zone_titre'][i][:6]
    for x in range(0, len(a)):
        a[x] = a[x].strip("\n\n")
        a[x] = a[x].replace('\\n\\n                                        ', '')
        a[x] = a[x].replace('\n', '')
        a[x] = a[x].replace('\\n                                    \\n', '')
        a[x] = a[x].replace('\\n', '')
        a[x] = a[x].replace('  ', '')
    b = df['Zone_contenu'][i][:6]
    for x in range(0, len(b)):
        b[x] = b[x].strip("\n\n")
        b[x] = b[x].replace('\\n\\n                                        ', '')
        b[x] = b[x].replace('\n', '')
        b[x] = b[x].replace('\\n                                    \\n', '')
        b[x] = b[x].replace('\\n', '')
        b[x] = b[x].replace('  ', '')

    dict_test = {
        'Name' : df['Name'][i],
        'Distributeur' : None,
        'Développeur' : None, 
        'Éditeur' : None, 
        'Franchise' : None, 
    }

    dict_zone = {}

    for x,y in zip(a,b):
        if x in dict_test.keys():
            dict_a = {x:y}
            dict_zone = dict(dict_zone, **dict_a)

    dict_test = dict(dict_test, **dict_zone)

    zone_dico.append(dict_test)
    

dico_df = pd.DataFrame.from_dict(zone_dico)

In [ ]:
#Merge des 2 sous-datasets pour obtenir le dataset final issu du 2ème scraping
df_GK_02 = df.drop(['Zone_titre', 'Zone_contenu'], axis=1)
df_GK_02 = df_GK_02.merge(right=dico_df,on='Name',how='left')

In [ ]:
df_GK_02.to_csv('../data/01_GameSpy_Scraping_gk_02.csv')